<a href="https://colab.research.google.com/github/titika2013/adv_ml_made_2022/blob/main/made_ml_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
pip install jiwer

In [2]:
import numpy as np
import re

from jiwer import wer
import nltk
import tqdm

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Реализуйте базовый частотный метод по Шерлоку Холмсу:

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
-расшифруйте их таким частотным методом.


In [4]:
def prepare_text(text):
    text = text.lower()
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nltk_tokens = tokenizer.tokenize(text)
    return ' '.join(nltk_tokens)


In [5]:
alphabet = " абвгдеёжзийклмнопрстуфхцчшщъыьэюя"


# #  Caesar Cipher 
# def caesar_cipher(message, LETTERS=alphabet):
#     for key in range(len(LETTERS)):
#         translated = ''
#         for symbol in message:
#             if symbol in LETTERS:
#                 num = LETTERS.find(symbol)
#                 num = num - key
#                 if num < 0:
#                     num = num + len(LETTERS)
#                 translated = translated + LETTERS[num]
#             else:
#                 translated = translated + symbol
#     return (key, translated)


def random_cipher(text, alphabet=alphabet):
    np.random.seed(42)
    ch_r_dict = {}
    for i in range(len(alphabet)):
        
        ch_r_dict[alphabet[i]] = alphabet[np.random.permutation(len(alphabet))[i]]
    return ''.join([ch_r_dict.get(symbol, '') for symbol in text])


def decipher_by_f(text, orig_freqs, n=1):
    new_t_freqs = nltk.FreqDist(nltk.ngrams(text, n))
    text_freqs_sorted = sorted(new_t_freqs, key=new_t_freqs.get, reverse=True)
    preg_freqs_sorted = sorted(orig_freqs, key=orig_freqs.get, reverse=True)
    dict_vals = dict(list(zip(text_freqs_sorted, preg_freqs_sorted)))
    out_str = ""
    for i, val in enumerate(nltk.ngrams(text, n)):
        if i % n == 0:
            out_str += "".join(dict_vals[val])
        else:
            out_str += ""
    return out_str


def text_ciph(text, mapp):
    return ''.join(mapp[x] for x in text)


In [6]:
text_sample = """Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!

Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных.
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня"""

In [7]:
prep_text = prepare_text(text_sample)
characters_freq = nltk.FreqDist(nltk.ngrams(prep_text, 1))

In [8]:
#caesar_ciphered_text = caesar_cipher(prep_text)[1]
random_ciphered = random_cipher(prep_text)

In [9]:
#caesar_hypothesis = decipher_by_f(caesar_ciphered_text, characters_freq, n=1)
hypothesis = decipher_by_f(random_ciphered, characters_freq, n=1)

In [10]:
hypothesis

'оеи тити лноьт яалодьт рмнсои велтн да с пвовв андаоел ед вснаное лаки анлонсои о ивяпа сьтвоное да оел але рмооам тмвлоо днввн де кеаа оеи внвни лвввн л кеиедьо лотаое о таде о деяе да еотети до пнлв рмеяе внвеа доавеа веснмлосе реиваоселе анкнсииое аов ретвпво рермнсииое раяниеде ретделоое иавнмлосе сатьтное о твоное рме лаки велтн аа яамо сеаеоао оаки онв твони оеиетеи ресалн иаои с рьио дн реяоесьт сласьпдаи сеиас аасалн днлиатдов слат лсеот метдьт тмваеи истооиь о мвлиндн л ламеао оеале меондн каа рматолиесои лаи аа янл реасеиеоа реаднвеооое снл едалод текмьи оеи рмоиоаие метоили дн кмалнт дась лта оеаао кьое метоиоле сь оио киолонио оеи яооноаие оно давелтн лвиии о и де сматад ласам тии оади'

In [11]:
 #print("Caesar cipher Wer: {}".format(wer(prep_text, caesar_hypothesis)))
print("Random cipher Wer: {}".format(wer(prep_text, hypothesis)))

Random cipher Wer: 1.0


## 2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы.

- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [12]:
characters_freq_bi  = nltk.FreqDist( nltk.ngrams(prep_text , 2))

In [13]:
text_bi = decipher_by_f(random_ciphered, characters_freq_bi, n=2)
print(text_bi)

 нь жежеи ровыс илревые диваь сеетмох й исгдовпрумкокагомечьгуро п бтепрасневаь а черс сдуевне н в с нолил погойсапонио а ыховмоли р м с нь льльь ятовмоолкустн а о ны ня пови па лион в савитже ва вляте атоннома м в исе ссеру е т пначекотадяпрчадеег нпрбяе  к жгде зьдиимод пк р стлие  клио  н лозлуо тай ервыро па евне не ати ень сеетмо чс саа тадр иа  итея мапогдеся ть ктье даилмоу  дй е гиа ыхе дн ндувобол ейх ь тау еб чзаак васу вебяй  бвоомаввоатвевыпонидрь бр о дсла ниемуммоол баткоя  мдям авуммотн согеры стваь  бь  чс асе  мтаст ие  мыхсеой нй асгох о меитожгия тье очодан пат откь ых редаквох дуи ныя  мко рг  пат ооддяй сл да теы неодя тьс ойроан пнеа х сеетмоятегь а ь лий едныме бзаражеь  иуч


In [14]:
print("Random cipher Wer bigram: {}".format(wer(prep_text, text_bi)))


Random cipher Wer bigram: 1.0916666666666666


## 3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают.

Основная часть задания – в том, как можно их улучшить:

- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

In [15]:
# with open('/content/drive/MyDrive/ml_hw_3/AnnaKarenina.txt', 'r') as tx:
#     annaKarenina = tx.read()

# with open('/content/drive/MyDrive/ml_hw_3/WarAndPeace.txt', 'r') as tx:
#     warAndPeace = tx.read()

# mega_text = warAndPeace + annaKarenina 
# characters_freq_bi  = nltk.FreqDist( nltk.ngrams(mega_text , 2))

In [16]:
def tune_parameters(ciph_text, characters_freq, orig_alphabet=alphabet, new_alphabet=alphabet, true_text=prep_text,
                    threshold=0.83, n=2):
    best_wer = np.inf
    for i in range(1, 1000):
        mcmc_map_dict = mapping_mcmc(ciph_text, characters_freq, list(orig_alphabet), 50, 200,
                                     new_alphabet=list(new_alphabet), rand_seed=i, n=n)
        text_mcmc = text_ciph(ciph_text, mcmc_map_dict)
        print("Wer: {}".format(wer(true_text, text_mcmc)), " random seed: :", i)
        if wer(true_text, text_mcmc) < best_wer:
            best_wer = wer(true_text, text_mcmc)
            best_param = text_mcmc
            best_rand = i
        if best_wer < threshold:
            break
    return best_wer, best_rand


In [17]:
def log_likelyhood_ngram(text, mapp, n, orig_ngram):
    log_likelyhood = 0
    text = text_ciph(text, mapp)
    text_freqs = nltk.FreqDist(nltk.ngrams(text, n))
    for ngram, freq in text_freqs.items():
        if orig_ngram[ngram] != 0:
            log_likelyhood += np.log(orig_ngram[ngram]) * freq
    return log_likelyhood


def new_dict(dict_val, rand_seed=None):
    dict_val = dict_val.copy()
    # np.random.seed(rand_seed)
    for _ in range(1):
        f, s = np.random.choice(list(dict_val.keys()), 2, replace=True)
        dict_val[f], dict_val[s] = dict_val[s], dict_val[f]
    return dict_val


def mhs(text, mapp, n_iter, orig_ngram, n, rand_seed=None):
    # np.random.seed(rand_seed)
    for i in range(n_iter):
        n_mapp = new_dict(mapp, rand_seed=rand_seed)
        likelyhood = log_likelyhood_ngram(text, n_mapp, n, orig_ngram)
        log_likelyhood = log_likelyhood_ngram(text, mapp, n, orig_ngram)
        if np.random.uniform() < min(1, np.exp(likelyhood - log_likelyhood)) and i < n_iter - 1:
            mapp = n_mapp
    return mapp, log_likelyhood


def mapping_mcmc(encoded_text, freqs_dict, alphabet, epochs, iters, new_alphabet=None, rand_seed=1, n=2):
    np.random.seed(rand_seed)
    if new_alphabet:
        new_map_dict = dict(list(zip(new_alphabet,
                                     np.random.permutation(alphabet))))
    else:
        new_map_dict = dict(list(zip(list(alphabet), np.random.permutation(list(alphabet)))))
    cur_map_dict, best_likelyhood = mhs(encoded_text, new_map_dict, iters, freqs_dict, n, rand_seed=rand_seed)
    best_map_dict = cur_map_dict
    for _ in tqdm.tqdm(range(epochs), desc="training..."):
        cur_map_dict, cur_likelyhood = mhs(encoded_text, cur_map_dict, iters, freqs_dict, n, rand_seed=rand_seed)
        if cur_likelyhood > best_likelyhood:
            best_likelyhood = cur_likelyhood
            best_map_dict = cur_map_dict
    return best_map_dict


In [18]:
best_wer, best_rand_val = tune_parameters(random_ciphered, characters_freq_bi, alphabet, alphabet, prep_text, 0.84)

training...: 100%|██████████| 50/50 [00:24<00:00,  2.00it/s]


Wer: 0.9833333333333333  random seed: : 1


training...: 100%|██████████| 50/50 [00:24<00:00,  2.08it/s]


Wer: 0.9666666666666667  random seed: : 2


training...: 100%|██████████| 50/50 [00:24<00:00,  2.04it/s]

Wer: 0.8333333333333334  random seed: : 3


In [19]:
mcmc_dict_map = mapping_mcmc(random_ciphered, characters_freq_bi, list(alphabet), 50, 300, rand_seed=best_rand_val)
text_ciph(random_ciphered, mcmc_dict_map)

training...: 100%|██████████| 50/50 [00:36<00:00,  1.38it/s]


'мои лили самыл гесмныл правми косла не в ткмкк еанемос он кваеамо седи еасмавми м икгте вылкмамо не мос есо прммер лрксмм накка но доее мои какаи сккка с доионым смлемо м лено м ного не омлоли нм таск прого какое нмекое коварсмво поикемвосо еадавиимо емк полкткм поправиимо пегаионо полносммо иекарсмво велыламо м лкмамо про седи косла ее герм воеомем меди мак лкмаи моиолои повеса иеми в пыим на погмовыл всевытнеи воиев еевеса насиелнмк всел свомл ролныл лркеои ивлммиы м рксиана с сероем моесо романа дее прелмсиовми сеи ее гас поевоиоме поенакомммо вас онесмн лодрыи мои прмимеио ролмиси на дресал невы сле моеем дымо ролмимсо вы мим димсмаим мои гммамеио мам некосла скиии м и но врелен север лии мени'

## 4. Расшифруйте сообщение:

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏



In [20]:
with open('/content/drive/MyDrive/ml_hw_3/AnnaKarenina.txt', 'r') as tx:
    annaKarenina = tx.read()

with open('/content/drive/MyDrive/ml_hw_3/WarAndPeace.txt', 'r') as tx:
    warAndPeace = tx.read()

mega_text = warAndPeace + annaKarenina
# возьмем весь текст
characters_freq_bi_max = nltk.FreqDist(nltk.ngrams(mega_text, 2))

In [21]:
text_ciphered = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'


In [22]:
enc_letters = ''.join(set(text_ciphered))
alphabet_new = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'[:len(enc_letters)]
translate = str.maketrans(enc_letters, alphabet_new)
new_text_ciphered = text_ciphered.translate(translate)
print("Переданное на другой алфавит: ", new_text_ciphered)

Переданное на другой алфавит:  тъцйкчскчйойзткжнщшбцфжсекйцйкиназйкжнщшбцфжсекзтхъзкркузнгнкъннпмтжйдкхнзнщсекцтгхнкищнайзбзфкъхнщтткчътгнкчскчъткъотцбцйкищбчйцфжнкйкинцрайзткшбхъйшбцфжсекпбццкёбкинъцтожтткатзчтщзнткёбобжйткхрщъбклнздкхнжтажнкдкжйатгнкжткнптмбв


In [23]:
true_text = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"

In [24]:
best_wer_2, best_rand_val_2 = tune_parameters(new_text_ciphered, characters_freq_bi_max, alphabet, alphabet, true_text,
                                              0.25)


training...: 100%|██████████| 50/50 [00:12<00:00,  4.03it/s]


Wer: 0.9714285714285714  random seed: : 1


training...: 100%|██████████| 50/50 [00:12<00:00,  4.00it/s]


Wer: 0.8571428571428571  random seed: : 2


training...: 100%|██████████| 50/50 [00:12<00:00,  3.98it/s]


...

Wer: 0.42857142857142855  random seed: : 11


training...: 100%|██████████| 50/50 [00:12<00:00,  4.02it/s]


Wer: 0.9714285714285714  random seed: : 12


training...: 100%|██████████| 50/50 [00:12<00:00,  4.02it/s]

Wer: 0.22857142857142856  random seed: : 13


In [25]:
mcmc_map_dict = mapping_mcmc(new_text_ciphered, characters_freq_bi_max, list(alphabet), 50, 200,
                             new_alphabet=list(alphabet), rand_seed=best_rand_val_2)
text_bi = text_ciph(new_text_ciphered, mcmc_map_dict)

training...: 100%|██████████| 50/50 [00:12<00:00,  3.98it/s]


In [26]:
text_bi

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

In [27]:
print("Wer: {}".format(wer(true_text, text_bi)))

Wer: 0.22857142857142856


## 5. Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше?

Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [28]:
mega_text = warAndPeace + annaKarenina
characters_freq_three = nltk.FreqDist(nltk.ngrams(mega_text, 3))

In [29]:
best_wer_3, best_rand_val_3 = tune_parameters(new_text_ciphered, characters_freq_three, alphabet, alphabet, true_text,
                                              0.1, n=3)

training...: 100%|██████████| 50/50 [00:13<00:00,  3.71it/s]


Wer: 1.0  random seed: : 1


training...: 100%|██████████| 50/50 [00:16<00:00,  3.05it/s]


Wer: 0.2  random seed: : 2


training...: 100%|██████████| 50/50 [00:14<00:00,  3.35it/s]

Wer: 0.0  random seed: : 3


In [30]:
mcmc_map_dict_3 = mapping_mcmc(new_text_ciphered, characters_freq_three, list(alphabet), 50, 200,
                               new_alphabet=list(alphabet), rand_seed=best_rand_val_3, n=3)
text_three = text_ciph(new_text_ciphered, mcmc_map_dict_3)


training...: 100%|██████████| 50/50 [00:15<00:00,  3.32it/s]


In [31]:
text_three

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'

In [32]:
print("Wer: {}".format(wer(true_text, text_three)))

Wer: 0.0
